In [2]:
import re  # For preprocessing
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import pandas as pd
import os
os.listdir()
print(os.listdir("Data/"))#Contents of the Data folder

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

['cz_item_category', 'item_content_cz', 'item_content_filtered (with_duplicates).csv', 'item_content_filtered_no_duplicates.csv', 'item_content_viewed_sid.csv', 'item_cz', 'item_list.csv', 'list_category', 'material_attributes_cz', 'product_views_cz', 'product_views_cz_filtered.csv', 'product_views_unified.csv', 'sentences.csv']


In [34]:
dataFile = "Data/product_views_unified.csv"
number_rows = 30000000
data_2020 = pd.read_csv(dataFile,nrows=number_rows)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]


In [35]:
data_2020['product_id'] = data_2020.loc[:,'product_id'].map(str)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]
user_id_anon
645         [655028, 1338684, 100018643178, 100018644575, ...
788         [100016425290, 452568001, 1086821, 1403084, 14...
871         [551806, 1305893, 1293084, 1389660, 1510064, 1...
1345        [100018205007, 100018206462, 100018206445, 100...
2981        [1480352, 1419661, 1105542, 1419654, 666213, 1...
                                  ...                        
62563472    [1349506, 1349510, 1349506, 1397954, 1490283, ...
62563934    [100016141442, 100016143151, 100016145005, 100...
62565140    [1236035, 100002172

In [ ]:
sentences = data_2020.groupby('user_id_anon')['product_id'].apply(list)
print(sentences)

In [36]:
most_viewed_element = data_2020["product_id"].value_counts().idxmax()
print(most_viewed_element)

100024602647


In [37]:
import multiprocessing

from gensim.models import Word2Vec
import numpy as np

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

# w2v_model = Word2Vec(min_count=10,
#                      window=2,
#                      size=300,
#                      sample=6e-5, 
#                      alpha=0.05, 
#                      min_alpha=0.001, 
#                      negative=20,
#                      workers=cores-1)

In [38]:
dataFile = "Data/product_views_unified.csv"
test_rows = 31502086 - number_rows
test_data = pd.read_csv(dataFile,names=["user_id_anon","product_id"],skiprows=number_rows,nrows=1000000)
test_data['product_id'] = test_data.loc[:,'product_id'].map(str)
test_set = test_data.groupby('user_id_anon')['product_id'].apply(list)
print(test_set)

user_id_anon
538256      [810546003, 810546003, 810546003, 810546003, 8...
889903      [100019753972, 100019305254, 100019974986, 100...
1478904     [1027666, 660649, 1027666, 1033245, 888073, 10...
1701672     [100020173950, 100029044379, 100029488800, 135...
4631801     [100016830262, 100016830262, 1223934, 10001683...
7127098     [1369991002, 1421387001, 1421387001, 116221900...
11350299    [100011304333, 1044888, 100013838783, 10001515...
15351895    [100018404773, 1247174, 1107042, 100015799314,...
17423924    [632960, 448602, 602001, 448602, 116350, 11095...
17618864    [969804001, 100019839747, 1273164, 1124037001,...
17685842    [996042, 492205, 1034216002, 1034216002, 10342...
21410504    [100016284804, 100011744558, 100004210791, 133...
23126675    [100022944375, 100022675574, 100017721476, 100...
25653713    [1102404004, 1102404004, 870690, 1318363, 1102...
27428334    [1416624, 100019173290, 489961, 726649, 100004...
31899327    [789166, 131864, 100012106631, 1049771, 76049

In [39]:
import itertools

parm_dict = {'size':(100,200,300),'window':(2,5),'min_count':(5,10),'alpha':(0.03,0.02,0.01)}

def cust_param_search(parm_dict,sentences,test_set,most_viewed_element):
    score_best, parm_best = 0,()
   
    size,window, min_count, alpha = [tup for k,tup in parm_dict.items()] # Individual parm tuples

    parm_combo = list(itertools.product(size,window, min_count, alpha)) # Create all combinations

    for parms in parm_combo:
        s, w, m, a = parms
        print (parms)
        model = Word2Vec(min_count=m,
                         window=w,
                         size=s,
                         alpha=a,
                         workers=cores-1)
        model.build_vocab(sentences, progress_per=5000)
        model.train(sentences, total_examples=model.corpus_count, epochs=10, report_delay=1)

        model.init_sims(replace=True)
        similarity = []
        for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
            simil = []
            for j in i:
                if j in model.wv.vocab and j != most_viewed_element:
#                     print(model.wv.similarity(most_viewed_element,j))
                    simil.append(model.wv.similarity(most_viewed_element,j))
            if len(simil)>0:
                similarity.append(sum(simil)/len(simil))
        
        score = sum(similarity)/len(similarity)
        print(score)

        if score > score_best:
            score_best = score
            parm_best = parms          
    print("Best score -",score_best, "Best parms - ",parm_best)

In [40]:
cust_param_search(parm_dict,sentences,test_set,most_viewed_element)

INFO - 13:59:53: collecting all words and their counts
INFO - 13:59:53: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:59:53: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types


(100, 2, 5, 0.05)


INFO - 13:59:53: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 13:59:54: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 13:59:54: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 13:59:54: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 13:59:54: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 13:59:54: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 13:59:54: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 13:59:54: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 13:59:54: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 13:59:54: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 13:59:54: PROGRE

0.5439323033122107
(100, 2, 5, 0.025)


INFO - 14:00:40: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 14:00:40: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 14:00:41: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 14:00:41: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 14:00:41: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 14:00:41: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 14:00:41: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 14:00:41: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 14:00:41: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 14:00:41: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 14:00:41: PROGRE

...

0.8261527332211448
(300, 5, 10, 0.05)


INFO - 14:14:22: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 14:14:22: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 14:14:22: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 14:14:22: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 14:14:22: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 14:14:23: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 14:14:23: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 14:14:23: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 14:14:23: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 14:14:23: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 14:14:23: colle

0.3520167415895233
(300, 5, 10, 0.025)


INFO - 14:14:54: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 14:14:54: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 14:14:54: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 14:14:54: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 14:14:55: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 14:14:55: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 14:14:55: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 14:14:55: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 14:14:55: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 14:14:55: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 14:14:55: colle

0.7629580381573797
Best score - 0.8351309505773115 Best parms -  (200, 5, 5, 0.025)
